# import

In [1]:
import json
import os
import re
from pprint import pprint

from soynlp.normalizer import repeat_normalize

In [2]:
def dump_jsonl(data, output_path, append=False):
    mode = "a+" if append else "w"
    with open(output_path, mode, encoding="utf-8") as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + "\n")


def load_jsonl(input_path) -> list:
    data = []
    with open(input_path, "r", encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line.rstrip("\n|\r")))
    return data

In [3]:
os.chdir("/home/jh/documents")

vocab_size = 24000

# load data

In [4]:
def processing(text):
    text = re.sub(r" +", r" ", text.strip())
    text = re.sub(r"(.{8,}?)\1+", r"\1", text)
    text = re.sub(
        r"[^ ㄱ-ㅎㅏ-ㅣ가-힣A-Za-z0-9~!@#$%\^\&*\(\)_\+\-=\[\]{},\./<>\?]", r"", text
    )
    text = re.sub(r"http.+", r"[URL]", text)

    return text.strip()

In [5]:
data = load_jsonl("datasets/git_category.json")

In [6]:
len(data), data[0]

(198582,
 {'content': '서울박정호 SKT 대표이사가 25일 서울 중구 을지로 SKT타워에서 열린 SK텔레콤 제37기 정기 주주총회에 참석하고 있다SK텔레콤이 주주가치 제고를 위해 한국거래소에 자사주 869만주 소각을 반영한 변경상장을 완료했다고 14일 밝혔다 발행주식 총수의약 2조6000억원규모다 이로써 전날 기준 SK텔레콤의 발행주식 총수는 기존 8075만주에서 7206만주로 줄어들었다앞서 SK텔레콤은 지난 4일 올해 인적분할에 앞서 기업가치주주가치 제고를 위해 기존 자사주를 사실상 전량 소각한다고 발표한 바 있다 이번 자사주 소각으로 발행주식 총수가 감소하면서 기존 주주들의 지분율이 모두 상승했다SK텔레콤은 분할 후 기업가치가 올라갈 것으로 전망하는 증권업계 전반의 시각을 감안하면 기업 펀더멘털 변동없이 주식 수만 줄어든 상황이라며 자사주 소각 전보다 주식가치 상승 여력이 더 커졌다고 분석했다SK텔레콤은 발행주식 총수가 줄었음에도 전날 시가총액이 약 22조5000억원을 기록해 주주총회에서 기업구조인적분할 개편을 공식화했던 지난 3월25일 시가총액약 20조5000억원보다 10가량 늘었다고 설명했다 주가도 연초 대비 30 이상 상승했다고 덧붙였다한편 SK텔레콤은 상반기 내 이사회 의결을 거쳐 오는 10월 주주총회 11월 인적분할 법인의 재상장을 통해 인적분할 절차를 끝낼 계획이다',
  'category': '[19]'})

In [7]:
data[0]["content"]

'서울박정호 SKT 대표이사가 25일 서울 중구 을지로 SKT타워에서 열린 SK텔레콤 제37기 정기 주주총회에 참석하고 있다SK텔레콤이 주주가치 제고를 위해 한국거래소에 자사주 869만주 소각을 반영한 변경상장을 완료했다고 14일 밝혔다 발행주식 총수의약 2조6000억원규모다 이로써 전날 기준 SK텔레콤의 발행주식 총수는 기존 8075만주에서 7206만주로 줄어들었다앞서 SK텔레콤은 지난 4일 올해 인적분할에 앞서 기업가치주주가치 제고를 위해 기존 자사주를 사실상 전량 소각한다고 발표한 바 있다 이번 자사주 소각으로 발행주식 총수가 감소하면서 기존 주주들의 지분율이 모두 상승했다SK텔레콤은 분할 후 기업가치가 올라갈 것으로 전망하는 증권업계 전반의 시각을 감안하면 기업 펀더멘털 변동없이 주식 수만 줄어든 상황이라며 자사주 소각 전보다 주식가치 상승 여력이 더 커졌다고 분석했다SK텔레콤은 발행주식 총수가 줄었음에도 전날 시가총액이 약 22조5000억원을 기록해 주주총회에서 기업구조인적분할 개편을 공식화했던 지난 3월25일 시가총액약 20조5000억원보다 10가량 늘었다고 설명했다 주가도 연초 대비 30 이상 상승했다고 덧붙였다한편 SK텔레콤은 상반기 내 이사회 의결을 거쳐 오는 10월 주주총회 11월 인적분할 법인의 재상장을 통해 인적분할 절차를 끝낼 계획이다'

In [8]:
def gen():
    for row in data:
        yield processing(row["content"])

In [9]:
user_defined_symbols = [
    "[PAD]",
    "[UNK]",
    "[CLS]",
    "[SEP]",
    "[MASK]",
    "[BOS]",
    "[EOS]"
]
# user_defined_symbols += [f"[UNK{i}]" for i in range(10)]
unused_token_num = 100
unused_list = [f"[UNUSED{i}]" for i in range(100)]
whole_user_defined_symbols = user_defined_symbols + unused_list

pprint(whole_user_defined_symbols)

['[PAD]',
 '[UNK]',
 '[CLS]',
 '[SEP]',
 '[MASK]',
 '[BOS]',
 '[EOS]',
 '[UNUSED0]',
 '[UNUSED1]',
 '[UNUSED2]',
 '[UNUSED3]',
 '[UNUSED4]',
 '[UNUSED5]',
 '[UNUSED6]',
 '[UNUSED7]',
 '[UNUSED8]',
 '[UNUSED9]',
 '[UNUSED10]',
 '[UNUSED11]',
 '[UNUSED12]',
 '[UNUSED13]',
 '[UNUSED14]',
 '[UNUSED15]',
 '[UNUSED16]',
 '[UNUSED17]',
 '[UNUSED18]',
 '[UNUSED19]',
 '[UNUSED20]',
 '[UNUSED21]',
 '[UNUSED22]',
 '[UNUSED23]',
 '[UNUSED24]',
 '[UNUSED25]',
 '[UNUSED26]',
 '[UNUSED27]',
 '[UNUSED28]',
 '[UNUSED29]',
 '[UNUSED30]',
 '[UNUSED31]',
 '[UNUSED32]',
 '[UNUSED33]',
 '[UNUSED34]',
 '[UNUSED35]',
 '[UNUSED36]',
 '[UNUSED37]',
 '[UNUSED38]',
 '[UNUSED39]',
 '[UNUSED40]',
 '[UNUSED41]',
 '[UNUSED42]',
 '[UNUSED43]',
 '[UNUSED44]',
 '[UNUSED45]',
 '[UNUSED46]',
 '[UNUSED47]',
 '[UNUSED48]',
 '[UNUSED49]',
 '[UNUSED50]',
 '[UNUSED51]',
 '[UNUSED52]',
 '[UNUSED53]',
 '[UNUSED54]',
 '[UNUSED55]',
 '[UNUSED56]',
 '[UNUSED57]',
 '[UNUSED58]',
 '[UNUSED59]',
 '[UNUSED60]',
 '[UNUSED61]',
 '[UNUSED

# train

In [10]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece

bert_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

In [11]:
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase, NFD, StripAccents

bert_tokenizer.normalizer = normalizers.Sequence([NFD(), Lowercase(), StripAccents()])

In [12]:
from tokenizers.pre_tokenizers import Whitespace

bert_tokenizer.pre_tokenizer = Whitespace()

In [13]:
from tokenizers.processors import TemplateProcessing

bert_tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[(t, i) for i, t in enumerate(user_defined_symbols)],
)

In [14]:
from tokenizers.trainers import WordPieceTrainer

trainer = WordPieceTrainer(
    vocab_size=vocab_size,
    special_tokens=whole_user_defined_symbols,
)
bert_tokenizer.train_from_iterator(gen(), trainer)
# It took 219m 56.8s

In [15]:
output = bert_tokenizer.encode("테스트용인데 잘 되는거같아?")
print(output.ids)

bert_tokenizer.decode(output.ids)

[2, 5235, 20889, 589, 1339, 2146, 619, 1853, 467, 1, 3]


'테스트 ##용인 ##데 잘 되는 ##거 ##같 ##아'

In [16]:
from tokenizers import decoders

bert_tokenizer.decoder = decoders.WordPiece()
bert_tokenizer.decode(output.ids)

'테스트용인데 잘 되는거같아'

# convert transformers tokenizer and save

In [17]:
from transformers import ElectraTokenizerFast


fast_tokenizer = ElectraTokenizerFast(tokenizer_object=bert_tokenizer)

/home/jh/.pyenv/versions/3.8.15/envs/jupyter/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
fast_tokenizer.pad_token = "[PAD]"
fast_tokenizer.unk_token = "[UNK]"
fast_tokenizer.cls_token = "[CLS]"
fast_tokenizer.sep_token = "[SEP]"
fast_tokenizer.mask_token = "[MASK]"
fast_tokenizer.bos_token = "[BOS]"
fast_tokenizer.eos_token = "[EOS]"

special_tokens_dict = {"additional_special_tokens": user_defined_symbols}
fast_tokenizer.add_special_tokens(special_tokens_dict)

0

In [19]:
fast_tokenizer.decode(fast_tokenizer.encode("테스트용"))

2022-11-30 14:19:44.308949: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 14:19:44.421862: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-30 14:19:44.913722: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-30 14:19:44.913798: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

'[CLS] 테스트용 [SEP]'

In [20]:
fast_tokenizer.save_pretrained("etc/DialogWordPiece")

('etc/DialogWordPiece/tokenizer_config.json',
 'etc/DialogWordPiece/special_tokens_map.json',
 'etc/DialogWordPiece/vocab.txt',
 'etc/DialogWordPiece/added_tokens.json',
 'etc/DialogWordPiece/tokenizer.json')

In [21]:
t = ElectraTokenizerFast.from_pretrained("etc/DialogWordPiece")

t(
    [["테스트용", "잘 되나"]],
    max_length=10,
    padding=True,
)

/home/jh/.pyenv/versions/3.8.15/envs/jupyter/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2267: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


{'input_ids': [[2, 5235, 483, 3, 1339, 642, 448, 3]], 'token_type_ids': [[0, 0, 0, 0, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1]]}